In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn

import matplotlib.pyplot as plt

# Selecting training and validation data
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.feature_extraction import FeatureHasher
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.feature_selection import mutual_info_regression

# ML Models
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

from xgboost import XGBRegressor

import itertools

## Import data

In [2]:
users_data = pd.read_json("proper_data/users2.json")
deliveries_data = pd.read_json("proper_data/deliveries2.json")
events_data = pd.read_json("proper_data/sessions2.json")
products_data = pd.read_json("proper_data/products2.json")

In [3]:
users_data

,user_id,name,city,street
0,102,Aurelia Malon,Police,pl. Brzoskwiniowa 11/53
1,103,Mateusz Kobel,Police,al. Wrocławska 10
2,104,Radosław Ratka,Mielec,pl. Nowa 89/04
3,105,Anastazja Oszust,Szczecin,ul. Częstochowska 80
4,106,Sylwia Nurek,Szczecin,al. Wiosenna 72
...,...,...,...,...
195,297,Andrzej Kreczmer,Szczecin,ul. Zdrojowa 43/11
196,298,Natan Świadek,Mielec,pl. Staffa 801
197,299,Jeremi Kajstura,Konin,pl. Wiklinowa 022
198,300,Gustaw Cofała,Gdynia,ulica Krakowska 847


In [4]:
deliveries_data

,purchase_id,purchase_timestamp,delivery_timestamp,delivery_company
0,20001,2021-03-26T06:19:44,2021-03-29T06:23:48,620
1,20002,2020-12-03T08:25:48,2020-12-07T08:27:30,360
2,20003,2021-03-04T02:46:03,2021-03-07T02:49:30,360
3,20004,2021-03-03T19:12:38,2021-03-06T19:13:32,620
4,20005,2020-11-04T23:03:37,2020-11-08T23:07:25,516
...,...,...,...,...
7176,27177,2021-03-27T12:44:44,2021-03-31T12:48:16,360
7177,27178,2021-02-25T01:12:12,2021-03-01T01:13:21,360
7178,27179,2021-02-28T09:50:14,2021-03-03T09:52:52,516
7179,27180,2021-03-04T04:53:21,2021-03-09T04:56:57,516


In [5]:
events_data

,session_id,timestamp,user_id,product_id,event_type,offered_discount,purchase_id
0,100001,2021-03-26 06:16:10,102,1281,VIEW_PRODUCT,0,NaN
1,100001,2021-03-26 06:18:47,102,1278,VIEW_PRODUCT,0,NaN
2,100001,2021-03-26 06:19:44,102,1278,BUY_PRODUCT,0,20001.0
3,100002,2020-12-03 08:25:15,102,1278,VIEW_PRODUCT,0,NaN
4,100002,2020-12-03 08:25:48,102,1278,BUY_PRODUCT,0,20002.0
...,...,...,...,...,...,...,...
40449,110787,2021-04-24 07:43:01,301,1004,VIEW_PRODUCT,0,NaN
40450,110787,2021-04-24 07:44:45,301,1013,VIEW_PRODUCT,0,NaN
40451,110787,2021-04-24 07:47:39,301,1006,VIEW_PRODUCT,0,NaN
40452,110787,2021-04-24 07:49:28,301,1007,VIEW_PRODUCT,0,NaN


In [6]:
products_data

,product_id,product_name,category_path,price
0,1001,Telefon Siemens Gigaset DA310,Telefony i akcesoria;Telefony stacjonarne,58.97
1,1002,Kyocera FS-1135MFP,Komputery;Drukarki i skanery;Biurowe urządzeni...,2048.50
2,1003,Kyocera FS-3640MFP,Komputery;Drukarki i skanery;Biurowe urządzeni...,7639.00
3,1004,Fallout 3 (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,49.99
4,1005,Szalone Króliki Na żywo i w kolorze (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,49.99
...,...,...,...,...
314,1315,Jabra Talk,Telefony i akcesoria;Akcesoria telefoniczne;Ze...,54.99
315,1316,Plantronics Voyager Legend,Telefony i akcesoria;Akcesoria telefoniczne;Ze...,249.00
316,1317,Plantronics Savi W740,Telefony i akcesoria;Akcesoria telefoniczne;Ze...,1303.96
317,1318,Plantronics Savi W710,Sprzęt RTV;Audio;Słuchawki,553.00


## Extracting features

In [7]:
max_timestamp = events_data.timestamp.max()

max_year_month = max_timestamp.year, max_timestamp.month
max_year_month

(2021, 4)

### "Indexing features"

In [8]:
events_data['year'] = events_data.timestamp.dt.year
events_data['month'] = events_data.timestamp.dt.month



years = list(events_data.year.unique())
months = list(events_data.month.unique())
user_ids = list(events_data.user_id.unique())

triplets = []
for triplet in itertools.product(years, months, user_ids):
    triplets.append(triplet)
    
processed_data = pd.DataFrame(triplets, columns=['year', 'month', 'user_id'])
processed_data.drop(processed_data[(processed_data['year'] == 2021) & (processed_data['month'] > 4)].index, inplace=True)

processed_data

,year,month,user_id
0,2021,3,102
1,2021,3,103
2,2021,3,104
3,2021,3,105
4,2021,3,106
...,...,...,...
4795,2020,10,297
4796,2020,10,298
4797,2020,10,299
4798,2020,10,300


### Number of all events per user per month

In [9]:
all_events = events_data.groupby(['user_id', 'year', 'month']).aggregate({"session_id": "count"}) \
    .rename(columns={"session_id": "all_sessions"}) \
    .reset_index()

### Number of buying events per user per month

In [10]:
buying_events = events_data[events_data['event_type'] == 'BUY_PRODUCT'] \
    .groupby(['user_id', 'year', 'month']) \
    .aggregate({"session_id": "count"}) \
    .rename(columns={"session_id": "buying_sessions"}) \
    .reset_index()

### Buying ratio

In [11]:
events_ratio = pd.merge(all_events, buying_events, how="left", on=["year", "month", "user_id"])

# all_events
# buying_events

events_ratio['buying_sessions'].fillna(0, inplace=True)

events_ratio['buying_ratio'] = round(events_ratio['buying_sessions'] / events_ratio['all_sessions'], 4)


# merging with processed_data
processed_data = pd.merge(processed_data, events_ratio.drop("all_sessions", axis=1), how='left', on=['year', 'month', 'user_id'])
processed_data['buying_sessions'].fillna(0, inplace=True)
processed_data['buying_ratio'].fillna(0, inplace=True)

### Constructing target variable - how much money the user spent per month

In [12]:
buying_sessions = events_data[events_data['event_type'] == "BUY_PRODUCT"]
deals = pd.merge(buying_sessions, products_data, how="left", on=['product_id'])
deals['final_price'] = deals['price'] * (1 - deals['offered_discount'] * 0.01)

monthly_deals = deals.groupby(['year', 'month', 'user_id']) \
    .aggregate({"final_price": "sum"}) \
    .rename(columns={"final_price": "money_monthly"}) \
    .reset_index()

processed_data = pd.merge(processed_data, monthly_deals, how='left', on=['year', 'month', 'user_id'])
processed_data['money_monthly'].fillna(0, inplace=True)

In [13]:
processed_data

,year,month,user_id,buying_sessions,buying_ratio,money_monthly
0,2021,3,102,5.0,0.2381,5863.1530
1,2021,3,103,25.0,0.2358,17784.3495
2,2021,3,104,5.0,0.1316,11720.3560
3,2021,3,105,6.0,0.3000,7220.2930
4,2021,3,106,14.0,0.1842,13131.9625
...,...,...,...,...,...,...
3195,2020,10,297,8.0,0.1778,3586.5420
3196,2020,10,298,2.0,0.2500,2600.1690
3197,2020,10,299,0.0,0.0000,0.0000
3198,2020,10,300,0.0,0.0000,0.0000


### Money spend in the previous month

In [14]:
user_dfs = []

look_back = 3
look_back_money = 3
for user_id in users_data.user_id:
    
    user_data = processed_data[processed_data.user_id == user_id].sort_values(['year', 'month'])
    
    
    for i in range(0, user_data.shape[0] - look_back):
        accumulated_value = 0
        for j in range(look_back):
            accumulated_value += user_data.loc[user_data.index[i + j], 'buying_sessions']
        
        user_data.loc[user_data.index[i + look_back], f'buying_sessions_MA{look_back}'] = np.round(accumulated_value / look_back, 1)
    
    
    for i in range(0, user_data.shape[0] - look_back):
        accumulated_value = 0
        for j in range(look_back):
            accumulated_value += user_data.loc[user_data.index[i + j], 'buying_ratio']
        
        user_data.loc[user_data.index[i + look_back], f'buying_ratio_MA{look_back}'] = np.round(accumulated_value / look_back, 3)
        
    
    for i in range(0, user_data.shape[0] - look_back_money):
        accumulated_value = 0
        for j in range(look_back_money):
            accumulated_value += user_data.loc[user_data.index[i + j], 'money_monthly']
        
        user_data.loc[user_data.index[i + look_back_money], f'money_monthly_MA{look_back_money}'] = np.round(accumulated_value / look_back_money, 1)
    
    user_data 
    
    if user_id % 100 == 0:
        print(user_id)
        
    user_data[f'buying_sessions_MA{look_back}'].fillna(0, inplace=True)
    user_data[f'buying_ratio_MA{look_back}'].fillna(0, inplace=True)
    user_data[f'money_monthly_MA{look_back_money}'].fillna(0, inplace=True)
    
    user_data['prev_month_spendings'] = user_data['money_monthly'].shift(1)
    user_data['prev_month_spendings'].fillna(0, inplace=True)
    
    user_data['prev_second_month_spendings'] = user_data['money_monthly'].shift(2)
    user_data['prev_second_month_spendings'].fillna(0, inplace=True)
    
    user_data['prev_third_month_spendings'] = user_data['money_monthly'].shift(3)
    user_data['prev_third_month_spendings'].fillna(0, inplace=True)
    
    
    user_data['buying_sessions_prev_month'] = user_data['buying_sessions'].shift(1)
    user_data['buying_sessions_prev_month'].fillna(0, inplace=True)
    
    user_data['buying_sessions_p_second_month'] = user_data['buying_sessions'].shift(2)
    user_data['buying_sessions_p_second_month'].fillna(0, inplace=True)
    
    user_data['buying_sessions_p_third_month'] = user_data['buying_sessions'].shift(3)
    user_data['buying_sessions_p_third_month'].fillna(0, inplace=True)
    
    
    user_data['buying_ratio_prev_month'] = user_data['buying_ratio'].shift(1)
    user_data['buying_ratio_prev_month'].fillna(0, inplace=True)
    
    user_data['buying_ratio_p_second_month'] = user_data['buying_ratio'].shift(2)
    user_data['buying_ratio_p_second_month'].fillna(0, inplace=True)
    
    user_data['buying_ratio_p_third_month'] = user_data['buying_ratio'].shift(3)
    user_data['buying_ratio_p_third_month'].fillna(0, inplace=True)
    
    user_dfs.append(user_data)
    

new_processed_data = pd.concat(user_dfs)
# processed_data = pd.concat(user_dfs)
# processed_data
new_processed_data

200
300


,year,month,user_id,buying_sessions,buying_ratio,money_monthly,buying_sessions_MA3,buying_ratio_MA3,money_monthly_MA3,prev_month_spendings,prev_second_month_spendings,prev_third_month_spendings,buying_sessions_prev_month,buying_sessions_p_second_month,buying_sessions_p_third_month,buying_ratio_prev_month,buying_ratio_p_second_month,buying_ratio_p_third_month
1400,2020,1,102,0.0,0.0000,0.0000,0.0,0.000,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0
1600,2020,2,102,0.0,0.0000,0.0000,0.0,0.000,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0
800,2020,3,102,0.0,0.0000,0.0000,0.0,0.000,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0
1800,2020,4,102,0.0,0.0000,0.0000,0.0,0.000,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0
2800,2020,5,102,0.0,0.0000,0.0000,0.0,0.000,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199,2020,12,301,0.0,0.0000,0.0000,0.0,0.000,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0
399,2021,1,301,0.0,0.0000,0.0000,0.0,0.000,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0
599,2021,2,301,3.0,0.1765,303.5325,0.0,0.000,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0
199,2021,3,301,11.0,0.1897,11304.8600,1.0,0.059,101.2,303.5325,0.0000,0.0,3.0,0.0,0.0,0.1765,0.0000,0.0


In [15]:
quantile_eighty_five = new_processed_data.groupby('user_id').aggregate({"money_monthly": 'sum'}).quantile(0.85).to_list()[0]

### Grouping users

In [16]:
user_lifetime_value = new_processed_data.groupby('user_id').aggregate({"money_monthly": "sum"}).rename(columns={"money_monthly": "lifetime_value"})

riches = []

for idx, row_data in user_lifetime_value.iterrows():
    
    row_data = dict(row_data)
    
    lf_value = row_data['lifetime_value']
    
    if lf_value > quantile_eighty_five:
        riches.append(1)
    else:
        riches.append(0)
        


# user_lifetime_value['is_rich'] = [not elem for elem in dummy]
user_lifetime_value['is_rich'] = riches
user_lifetime_value = user_lifetime_value.reset_index()

new_processed_data = pd.merge(new_processed_data, user_lifetime_value[['user_id', 'is_rich']], on='user_id', how='left')
new_processed_data

,year,month,user_id,buying_sessions,buying_ratio,money_monthly,buying_sessions_MA3,buying_ratio_MA3,money_monthly_MA3,prev_month_spendings,prev_second_month_spendings,prev_third_month_spendings,buying_sessions_prev_month,buying_sessions_p_second_month,buying_sessions_p_third_month,buying_ratio_prev_month,buying_ratio_p_second_month,buying_ratio_p_third_month,is_rich
0,2020,1,102,0.0,0.0000,0.0000,0.0,0.000,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0
1,2020,2,102,0.0,0.0000,0.0000,0.0,0.000,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0
2,2020,3,102,0.0,0.0000,0.0000,0.0,0.000,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0
3,2020,4,102,0.0,0.0000,0.0000,0.0,0.000,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0
4,2020,5,102,0.0,0.0000,0.0000,0.0,0.000,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,2020,12,301,0.0,0.0000,0.0000,0.0,0.000,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0
3196,2021,1,301,0.0,0.0000,0.0000,0.0,0.000,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0
3197,2021,2,301,3.0,0.1765,303.5325,0.0,0.000,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0
3198,2021,3,301,11.0,0.1897,11304.8600,1.0,0.059,101.2,303.5325,0.0000,0.0,3.0,0.0,0.0,0.1765,0.0000,0.0,0


### Masking the last 5 month

In [17]:
mask_twenty = (new_processed_data['year'] == 2020) & ((new_processed_data['month'] == 12) | (new_processed_data['month'] == 11))
mask_twenty_one = (new_processed_data['year'] == 2021)

totally_new_data = new_processed_data[mask_twenty | mask_twenty_one]
totally_new_data

,year,month,user_id,buying_sessions,buying_ratio,money_monthly,buying_sessions_MA3,buying_ratio_MA3,money_monthly_MA3,prev_month_spendings,prev_second_month_spendings,prev_third_month_spendings,buying_sessions_prev_month,buying_sessions_p_second_month,buying_sessions_p_third_month,buying_ratio_prev_month,buying_ratio_p_second_month,buying_ratio_p_third_month,is_rich
10,2020,11,102,4.0,0.1667,1892.2815,0.0,0.000,0.0,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0000,0.0000,0.0000,0
11,2020,12,102,5.0,0.2500,2658.0000,1.3,0.056,630.8,1892.2815,0.0000,0.0000,4.0,0.0,0.0,0.1667,0.0000,0.0000,0
12,2021,1,102,6.0,0.2400,4787.8560,3.0,0.139,1516.8,2658.0000,1892.2815,0.0000,5.0,4.0,0.0,0.2500,0.1667,0.0000,0
13,2021,2,102,6.0,0.2069,1521.9215,5.0,0.219,3112.7,4787.8560,2658.0000,1892.2815,6.0,5.0,4.0,0.2400,0.2500,0.1667,0
14,2021,3,102,5.0,0.2381,5863.1530,5.7,0.232,2989.3,1521.9215,4787.8560,2658.0000,6.0,6.0,5.0,0.2069,0.2400,0.2500,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,2020,12,301,0.0,0.0000,0.0000,0.0,0.000,0.0,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0000,0.0000,0.0000,0
3196,2021,1,301,0.0,0.0000,0.0000,0.0,0.000,0.0,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0000,0.0000,0.0000,0
3197,2021,2,301,3.0,0.1765,303.5325,0.0,0.000,0.0,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0000,0.0000,0.0000,0
3198,2021,3,301,11.0,0.1897,11304.8600,1.0,0.059,101.2,303.5325,0.0000,0.0000,3.0,0.0,0.0,0.1765,0.0000,0.0000,0


### Mutual information score

In [18]:
totally_new_data.drop(['year', 'month', 'user_id', 'buying_sessions', 'buying_ratio'], axis=1, inplace=True)

data_to_mi_score = totally_new_data.dropna(subset=['money_monthly'])

X = data_to_mi_score.copy()
y = X.pop("money_monthly")

# Label encoding for categoricals
for colname in X.select_dtypes("object"):
    X[colname], _ = X[colname].factorize()

# All discrete features should now have integer dtypes (double-check this before using MI!)
discrete_features = X.dtypes == int

def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

mi_scores = make_mi_scores(X, y, discrete_features)
mi_scores

/home/tomek/.local/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


buying_sessions_prev_month        0.399319
buying_sessions_MA3               0.329909
buying_ratio_prev_month           0.315329
money_monthly_MA3                 0.306254
buying_ratio_MA3                  0.302611
prev_month_spendings              0.302509
buying_sessions_p_second_month    0.186947
prev_second_month_spendings       0.168172
buying_ratio_p_second_month       0.158962
buying_ratio_p_third_month        0.115330
buying_sessions_p_third_month     0.106279
prev_third_month_spendings        0.095913
is_rich                           0.044325
Name: MI Scores, dtype: float64

## Training

In [19]:
data = totally_new_data

useful_features = ['is_rich', f'buying_sessions_MA{look_back}', f'money_monthly_MA{look_back_money}', 'buying_sessions_prev_month', 'buying_ratio_prev_month','prev_month_spendings']

data_to_train = data[useful_features + ['money_monthly']]
X = data_to_train[useful_features]

y = data_to_train['money_monthly']

### Linear Regression

In [20]:
linear_model = LinearRegression()

scores_linear = cross_val_score(
    linear_model,
    X=X,
    y=y,
    scoring='neg_mean_absolute_error',
    cv=5
)

print("Linear Regression MAE is: ", (-1) * scores_linear.mean())


print(f"Mean user spending: {np.mean(y)}")

Linear Regression MAE is:  3039.929222758264
Mean user spending: 3548.8187883333335


### Moving Average model

In [21]:
user_dfs = []

look_back = 2
for user_id in users_data.user_id:
    
    user_data = new_processed_data[new_processed_data.user_id == user_id].sort_values(['year', 'month'])
    

    for i in range(0, user_data.shape[0] - look_back):
        
        accumulated_value = 0
        for j in range(look_back):
            accumulated_value += user_data.loc[user_data.index[i + j], 'money_monthly']
        
        user_data.loc[user_data.index[i + look_back], f'SMA_{look_back}_before'] = np.round(accumulated_value / look_back, 1)

    if user_id % 100 == 0:
        print(user_id)
    
    user_data.fillna(0, inplace=True)
    user_dfs.append(user_data[['year', 'month', 'user_id', 'money_monthly', f'SMA_{look_back}_before']])

    
moving_average_data = pd.concat(user_dfs)

mask_twenty = (moving_average_data['year'] == 2020) & ((moving_average_data['month'] == 12) | (moving_average_data['month'] == 11))
mask_twenty_one = (moving_average_data['year'] == 2021)

# masking
moving_average_data = moving_average_data[mask_twenty | mask_twenty_one]


mae = mean_absolute_error(moving_average_data['money_monthly'], moving_average_data[f'SMA_{look_back}_before'])

print("Moving Average MAE: ", mae)

200
300
Moving Average MAE:  2861.198245


### Support Vector Regressor

In [22]:
svr_model = SVR(kernel='rbf', C=10000)

scores_svr = cross_val_score(
    svr_model,
    X=X,
    y=y,
    scoring='neg_mean_absolute_error',
    cv=5
)

print("SVR MAE is: ", (-1) * scores_svr.mean())

SVR MAE is:  2609.7856416694585


### Random Forest

In [23]:
rf_model = RandomForestRegressor(n_estimators=101, max_depth=10)

scores_rf = cross_val_score(
    rf_model,
    X=X,
    y=y,
    scoring='neg_mean_absolute_error',
    cv=10
)

print("Random Forest Regressor score is: ", (-1) * scores_rf.mean())

Random Forest Regressor score is:  2730.944450780481


### Gradient Boosting

In [24]:
gb_model = XGBRegressor(silent=1)

scores_gb = cross_val_score(
    gb_model,
    X=X,
    y=y,
    scoring='neg_mean_absolute_error',
    cv=10
)

print("XGBoost Regressor score is ", (-1) * scores_gb.mean())

[12:08:31] WARNING: ../src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[12:08:31] WARNING: ../src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[12:08:31] WARNING: ../src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[12:08:31] WARNI